In [6]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [12]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4f10336d31d9cd7f238af90c297b605f


In [13]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform (low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform (low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs

In [14]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_lngs)

In [15]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

594

In [16]:
# Create an empty list to hold the weather data. 
city_data = []

#Print the beginning of the logging.
print("Beginning Data Retrieval")
print("------------------------")

#Create Counters
record_count = 1
set_count = 1 

Beginning Data Retrieval
------------------------


In [21]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Cloudiness": city_clouds,
                          "Country": city_country,
                          "Date": city_date,
                          "Humidity": city_humidity,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 45 of Set 34 | taolanaro
City not found. Skipping...
Processing Record 46 of Set 34 | broome
Processing Record 47 of Set 34 | ushuaia
Processing Record 48 of Set 34 | albany
Processing Record 49 of Set 34 | katsuura
Processing Record 50 of Set 34 | mataura
Processing Record 51 of Set 34 | shibarghan
City not found. Skipping...
Processing Record 52 of Set 34 | hobart
Processing Record 53 of Set 34 | erzin
Processing Record 54 of Set 34 | punta arenas
Processing Record 55 of Set 34 | rikitea
Processing Record 56 of Set 34 | pakwach
City not found. Skipping...
Processing Record 57 of Set 34 | krasnoselkup
Processing Record 58 of Set 34 | bluff
Processing Record 59 of Set 34 | narsaq
Processing Record 60 of Set 34 | umuarama
Processing Record 61 of Set 34 | saint-philippe
Processing Record 62 of Set 34 | mecca
Processing Record 63 of Set 34 | clyde river
Processing Record 64 of Set 34 | ilulissat
Processing Record 65 of Set 34 | jamestown
Processing Record 66 of Set 34 | 

In [27]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Broome,42.2506,-75.8330,60.01,28,1,12.66,US,2021-04-23 21:06:35
1,Ushuaia,-54.8000,-68.3000,42.80,87,75,4.61,AR,2021-04-23 21:04:04
2,Albany,42.6001,-73.9662,60.01,22,5,4.00,US,2021-04-23 21:05:29
3,Katsuura,35.1333,140.3000,53.01,76,40,6.91,JP,2021-04-23 21:06:36
4,Mataura,-46.1927,168.8643,45.00,95,53,1.99,NZ,2021-04-23 21:06:36
5,Hobart,-42.8794,147.3294,51.80,71,75,8.05,AU,2021-04-23 21:06:16
6,Erzin,36.9559,36.2000,66.00,79,94,3.60,TR,2021-04-23 21:06:36
7,Punta Arenas,-53.1500,-70.9167,46.40,93,75,3.44,CL,2021-04-23 21:05:59
8,Rikitea,-23.1203,-134.9692,79.09,67,100,17.78,PF,2021-04-23 21:06:37
9,Krasnoselkup,65.7000,82.4667,29.57,88,90,17.22,RU,2021-04-23 21:06:37


In [28]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df_new = city_data_df[new_column_order]
city_data_df_new.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Broome,US,2021-04-23 21:06:35,42.2506,-75.8330,60.01,28,1,12.66
1,Ushuaia,AR,2021-04-23 21:04:04,-54.8000,-68.3000,42.80,87,75,4.61
2,Albany,US,2021-04-23 21:05:29,42.6001,-73.9662,60.01,22,5,4.00
3,Katsuura,JP,2021-04-23 21:06:36,35.1333,140.3000,53.01,76,40,6.91
4,Mataura,NZ,2021-04-23 21:06:36,-46.1927,168.8643,45.00,95,53,1.99
5,Hobart,AU,2021-04-23 21:06:16,-42.8794,147.3294,51.80,71,75,8.05
6,Erzin,TR,2021-04-23 21:06:36,36.9559,36.2000,66.00,79,94,3.60
7,Punta Arenas,CL,2021-04-23 21:05:59,-53.1500,-70.9167,46.40,93,75,3.44
8,Rikitea,PF,2021-04-23 21:06:37,-23.1203,-134.9692,79.09,67,100,17.78
9,Krasnoselkup,RU,2021-04-23 21:06:37,65.7000,82.4667,29.57,88,90,17.22


In [35]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")